In [1]:
import os
import sys
import ast
from datetime import datetime

import numpy as np
import pandas as pd
import torch
import timm
import torch.nn as nn
from PIL import Image
from torch import tensor
from torchvision.transforms import transforms, InterpolationMode
from tqdm.auto import tqdm

e:\mgr\.venv_windows\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DFPATH = 'models'
BACKBONE = "vit_large_patch14_dinov2.lvd142m"
DFVIT_WEIGHTS = os.path.join(DFPATH, 'deepfaune-vit_large_patch14_dinov2.lvd142m.v4.pt')

#### Setup

In [3]:
images = pd.read_csv('../megadetector_results.csv', index_col=0)
images['image_path'] = '../' + images['image_path']
images['bbox'] = images["bbox"].apply(
    lambda b: ast.literal_eval(b) if isinstance(b, str) else None)

In [4]:
BATCH_SIZE = 20
CROP_SIZE = 476  # default 182, has to be divisible by 14, 280? TODO

class_labels = [
    'bison', 'badger', 'ibex', 'beaver', 'red deer', 'golden jackal', 'chamois',
    'cat', 'goat', 'roe deer', 'dog', 'raccoon dog', 'fallow deer', 'squirrel',
    'moose', 'equid', 'genet', 'wolverine', 'hedgehog', 'lagomorph', 'wolf',
    'otter', 'lynx', 'marmot', 'micromammal', 'mouflon', 'sheep', 'mustelid',
    'bird', 'bear', 'porcupine', 'nutria', 'muskrat', 'raccoon', 'fox', 'reindeer',
    'wild boar', 'cow'
]


def crop_normalized_bbox(img: Image.Image, bbox: list[float]):
    """
    | img: PIL.Image opened image
    | bbox: list [x, y, w, h], normalized 0-1
    | returns cropped PIL.Image
    """
    W, H = img.size
    x, y, w, h = bbox

    left = int(x * W)
    top = int(y * H)
    right = int((x + w) * W)
    bottom = int((y + h) * H)

    return img.crop((left, top, right, bottom))


class Classifier:
    def __init__(self):
        self.model = Model()
        self.model.loadWeights(DFVIT_WEIGHTS)
        self.transforms = transforms.Compose([
            transforms.Resize(
                size=(CROP_SIZE, CROP_SIZE),
                interpolation=InterpolationMode.BICUBIC,
                antialias=None
            ),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=tensor([0.4850, 0.4560, 0.4060]),
                std=tensor([0.2290, 0.2240, 0.2250]))]
            )
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )

    def predictOnBatch(self, pil_images: list[Image.Image],
                       withsoftmax=True)-> np.ndarray:
        tensors = [self.transforms(im) for im in pil_images]
        batch = torch.stack(tensors).to(self.device)  # (B, C, H, W)

        with torch.inference_mode():
            outputs = self.model.predict(batch, withsoftmax=withsoftmax)

        return outputs


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = timm.create_model(
            BACKBONE, pretrained=False,
            num_classes=len(class_labels),
            dynamic_img_size=True
        )
        print(f"Using {BACKBONE}, in resolution {CROP_SIZE}x{CROP_SIZE}")

        self.n_classes = len(class_labels)
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        print("Using CUDA" if torch.cuda.is_available() else "Using CPU")
        self.to(self.device)

    def forward(self, input):
        return self.base_model(input)  

    def predict(self, x, withsoftmax=True):
        self.eval()
        with torch.inference_mode():
            x = x.to(self.device)
            logits = self.forward(x)
            return logits.softmax(dim=1) if withsoftmax else logits

    def loadWeights(self, path):
        try:
            params = torch.load(
                path, map_location=self.device, weights_only=False
            )
            args = params['args']

            if self.n_classes != args['num_classes']:
                raise Exception(
                    f"Checkpoint has {args['num_classes']} classes, but"
                    "model expects {self.n_classes}"
                )
            self.load_state_dict(params['state_dict'])
        except Exception as e:
            print(f"Can't load checkpoint model :\n {str(e)}", file=sys.stderr)
            raise e

classifier = Classifier()

Using vit_large_patch14_dinov2.lvd142m, in resolution 476x476
Using CUDA


### The loop

In [5]:
results = pd.DataFrame({'image': [], 'detected_animal': [], 'confidence': []})
batch = []
paths = []

for _, row in tqdm(images.iterrows(), total=len(images)):
    image_path = row['image_path']

    category = row['category']
    if category != 1:
        results.loc[len(results)] = [image_path, 'empty', 0]
        continue

    try:
        image = Image.open(image_path).convert("RGB")
        cropped_image = crop_normalized_bbox(image, row['bbox'])
    except Exception as e:
        print(f'Error in image {image_path}: {e}')
        continue

    paths.append(image_path)
    batch.append(cropped_image)

    if len(batch) == BATCH_SIZE:
        preds = classifier.predictOnBatch(batch)
        top_probs, class_indxes = preds.max(dim=1)
        confs = top_probs.cpu().numpy()
        detections = [class_labels[idx] for idx in class_indxes]

        batch_results = pd.DataFrame(
            {'image': paths, 'detected_animal': detections, 'confidence': confs})
        results = pd.concat([results, batch_results], ignore_index=True)
        batch = []
        paths = []

if len(batch) > 0:
    preds = classifier.predictOnBatch(batch)
    top_probs, class_indxes = preds.max(dim=1)
    confs = top_probs.cpu().numpy()
    detections = [class_labels[idx] for idx in class_indxes]

    batch_results = pd.DataFrame(
        {'image': paths, 'detected_animal': detections, 'confidence': confs})
    results = pd.concat([results, batch_results], ignore_index=True)

now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results.to_csv(f'../results/deepfaune/results_deepfaune_{now}.csv')

 24%|██▍       | 4431/18107 [34:40<2:02:53,  1.85it/s] 

Error in image ../../pictures/10_ŁUPKÓW/B/Późna jesień/1/2022-11-18 05-58-30.JPG: broken data stream when reading image file


 24%|██▍       | 4435/18107 [34:40<1:01:17,  3.72it/s]

Error in image ../../pictures/10_ŁUPKÓW/B/Późna jesień/1/2022-11-18 05-59-31.JPG: broken data stream when reading image file


 25%|██▍       | 4458/18107 [34:52<55:43,  4.08it/s]  

...

 31%|███▏      | 5662/18107 [44:09<40:47,  5.08it/s]  

Error in image ../../pictures/11_ZUBEŃSKO/B/Wczesna jesień/3/2022-10-02 21-42-15.JPG: broken data stream when reading image file


 45%|████▍     | 8122/18107 [1:03:11<28:00,  5.94it/s]  

Error in image ../../pictures/16_HUTA_KOMOROWSKA/B/Lato/1/2023-07-08 08-12-33.JPG: image file is truncated (22 bytes not processed)


100%|██████████| 18107/18107 [3:52:32<00:00,  1.30it/s]      


In [6]:
pd.set_option('display.max_colwidth', None)
results

,image,detected_animal,confidence
0,../../pictures/01_CZARNE/B/Lato/2/2023-07-18 02-03-54.JPG,empty,0.000000
1,../../pictures/01_CZARNE/B/Lato/2/2023-07-18 02-06-43.JPG,empty,0.000000
2,../../pictures/01_CZARNE/B/Lato/2/2023-07-25 18-36-14.JPG,empty,0.000000
3,../../pictures/01_CZARNE/B/Lato/2/2023-07-26 03-47-00.JPG,empty,0.000000
4,../../pictures/01_CZARNE/B/Lato/2/2023-07-26 03-47-13.JPG,empty,0.000000
...,...,...,...
18086,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-02-22 17-24-47.JPG,fox,0.983016
18087,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-02-23 12-08-08.JPG,roe deer,0.996249
18088,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-03-01 15-46-14.JPG,fox,0.845402
18089,../../pictures/30_SUCHEDNIÓW/R/Zima/3/2023-03-01 20-06-01.JPG,mustelid,0.981854


### Classify single animal

In [55]:
classifier = Classifier()

Using vit_large_patch14_dinov2.lvd142m with weights at models/deepfaune-vit_large_patch14_dinov2.lvd142m.v4.pt, in resolution 560x560
CUDA available


In [ ]:
cropped_tensor = torch.ones((1, 3, CROP_SIZE, CROP_SIZE))
cropped_tensor[0, :, :, :] =  classifier.preprocessImage(image)
scores = classifier.model.predict(cropped_tensor)
scores

array([[ 2.2498e-06,  2.9331e-06,  2.8294e-06,  3.9971e-06,  7.6311e-06,  9.9209e-06,  1.6024e-06,  0.00010547,  1.4596e-05,  1.0276e-05,  1.5422e-05,   1.076e-05,  2.0871e-06,  5.1336e-06,  1.1234e-05,   1.012e-06,  3.3339e-05,  5.4318e-06,  1.3989e-06,  7.7259e-06,  4.0932e-06,  2.5115e-06,     0.99964,
         8.8029e-07,   3.431e-06,  1.3685e-06,  1.8991e-06,  9.2729e-06,  6.2021e-06,  1.0418e-05,  1.8977e-06,  2.1094e-05,  8.4796e-07,  1.6212e-06,  1.8961e-05,  1.6373e-06,  8.7422e-06,  1.1793e-05]])

In [ ]:
output = pd.DataFrame({'species':class_labels, 'score':scores[0][:]})
output = output[output['score'] > 0.2]

In [ ]:
print(f"Animal detected: {output.loc[output['score'].idxmax(), 'species']}")

Wykryto zwierzę: lynx



(eog:16505): Gtk-WARNING **: 08:49:08.322: Theme file for DMZ-Black has no directories
